In [1]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import time
import pandas as pd
import requests

In [2]:
base = 'https://www.life-global.org'
url = 'https://www.life-global.org/allcourses?page=1'

def getSoup(uri):
    driver_path = 'C:/Users/kevin/OneDrive/Escritorio/SEPTIMO SEMESTRE/Semantic Web/ChromeDriver/chromedriver.exe'
    driver = webdriver.Chrome(executable_path=driver_path)
    driver.get(uri)
    time.sleep(3)
    html = driver.page_source
    driver.quit()
    soup = bs(html, 'html.parser')
    return soup
"""
Esta función estaba pensada para recuperar el html relevante a las lecciones de los cursos, pero desafortunadamente esa parte de lá página es
dinámica y funciona de manera extraña dado que no existe ningún botón ni enlace que podamos seguir. Posiblemente sea un script pero no pudimos obtener acceso
a ellos. Ante esto, nos dimos cuenta que la estructura de las lecciones de los cursos es la misma siempre, así que nos tomamos la libertad de guardarlos de manera manual.
"""
def getSoupAndFlex(uri):
    driver_path = 'C:/Users/kevin/OneDrive/Escritorio/SEPTIMO SEMESTRE/Semantic Web/ChromeDriver/chromedriver.exe'
    driver = webdriver.Chrome(executable_path=driver_path)
    driver.get(uri)
    html = driver.page_source
    soup = bs(html, 'html.parser')

    lessons_tab = soup.find('li', class_="ui-core-emotion-cache-34o57r")
    print(lessons_tab)
    lessons_tab.click()
    """
    lower = lessons_tab.descendants
    spa = ""
    for t in lower:
        if t.name == "span":
            spa = t
    print(spa)
    spa.click()
    """
    
    html2 = html = driver.page_source
    soupLesson = bs(html2, 'html.parser')
    driver.quit()
    
    return soup, soupLesson


In [3]:
def getCatAndLinks(soup, base):
    certificat = soup.find_all('h2', class_="ui-core-emotion-cache-1htcy9t")
    certifications = []
    for cer in certificat:
        certifications.append(cer.text)
    links = []
    linki = soup.find_all('span', class_="ui-core-emotion-cache-4233sn") 
    linki = linki[2:]
    linki = linki[:5]
    for link in linki:
        links.append(link.get("href"))
    cerlinks = [base+cer for cer in links]
    print(certifications)
    print(cerlinks)
    return certifications, cerlinks


In [4]:
def getCatInfo(names, links, base):
    courses = {}
    for n in names:
        courses[n] = []
    couLinks = {}
    for n in names:
        couLinks[n] = []
    i = 0
    for link in links:

        name = names[i]
        soup = getSoup(link)

        coursi = soup.find_all('h1', class_="ui-core-emotion-cache-g77xg1")
        temp = []
        for co in coursi:
            temp.append(co.text)
        courses[name] = temp

        linki = soup.find_all("span", class_="ui-core-emotion-cache-4233sn")
        temp = []
        for li in linki:
            temp.append(li.get("href"))
        temp = [base+li for li in temp]
        temp = temp[2:]
        temp = temp[:-16]
        couLinks[name] = temp

        i += 1
    return courses, couLinks




In [5]:
def getCourseAndLessonsInfo(dicNombres, dicLinks):
    cat = dicNombres.keys()
    coursesInfo = []
    lessonsInfo = []
    for ca in cat:
        links = dicLinks[ca]
        i = 0
        for co in dicNombres[ca]:
            dicto = {
                "Name":co,
                "Duration":"",
                "Pace":"",
                "Language":"",
                "Author":"",
                "Code":"",
                "Description":"",
                "Category":"",
                "Link":links[i]
            }
            coursesInfo.append(dicto)
            i +=1
    for j in range(0,len(coursesInfo)):
        cu = coursesInfo[j]
        soup = getSoup(cu["Link"])
        info = soup.find_all("div", class_ = "ui-core-emotion-cache-1ui1n6c")
        cu["Duration"] = info[0].text
        cu["Pace"] = info[1].text
        cu["Category"] = info[2].text
        cu["Language"] = info[3].text
        cu["Author"] = info[4].text
        cu["Code"] = info[5].text
        cu["Description"] = soup.find("div", class_ = "ui-core-emotion-cache-8gtjff").text

        #Aquí añadimos de manera manual las lecciones
        lesson1 = {
            "Type": "Introduction",
            "Name": "Story",
            "Course": cu["Name"]
        }

        lesson2 = {
            "Type": "Modules",
            "Name": "Bussiness Concept",
            "Course": cu["Name"]
        }

        lesson3 = {
            "Type": "Modules",
            "Name": "Technology Skills",
            "Course": cu["Name"]
        }

        lesson4 = {
            "Type": "Modules",
            "Name": "Survey",
            "Course": cu["Name"]
        }

        lesson5 = {
            "Type": "Modules",
            "Name": "Resources",
            "Course": cu["Name"]
        }

        lessonsInfo.append(lesson1)
        lessonsInfo.append(lesson2)
        lessonsInfo.append(lesson3)
        lessonsInfo.append(lesson4)
        lessonsInfo.append(lesson5)

    return coursesInfo, lessonsInfo


In [6]:
languageAll = ["https://www.life-global.org/allcourses?page=1", "https://www.life-global.org/fr/allcourses?page=1", "https://www.life-global.org/ar/allcourses?page=1", "https://www.life-global.org/hi/allcourses?page=1", "https://www.life-global.org/pt/allcourses?page=1", "https://www.life-global.org/es/allcourses?page=1", "https://www.life-global.org/zh-Hans/allcourses?page=1", "https://www.life-global.org/id/allcourses?page=1"]
languageBase = ["https://www.life-global.org/", "https://www.life-global.org/fr", "https://www.life-global.org/ar", "https://www.life-global.org/hi", "https://www.life-global.org/pt", "https://www.life-global.org/es", "https://www.life-global.org/zh-Hans","https://www.life-global.org/id"]

def getAll(urlAll, urlBase):

    soupAll = getSoup(urlAll)

    cate = getCatAndLinks(soupAll, urlBase)
    catenames = cate[0]
    catelinks = cate[1]

    catCourses = getCatInfo(catenames, catelinks, urlBase)
    courNames = catCourses[0]
    courLinks = catCourses[1]

    coursesLessons = getCourseAndLessonsInfo(courNames, courLinks)
    coursesLa = coursesLessons[0]
    lessonsLa = coursesLessons[1]

    return catenames, coursesLa, lessonsLa


In [7]:
df_categories = None
df_courses = None
df_lessons = None



language =  getAll(languageAll[1], languageBase[1])

if df_categories == None:
    df_categories = pd.DataFrame(language[0], columns=['Name'])
else:
    new = pd.DataFrame(language[0], columns=['Name'])
    df_categories = df_categories.append(new, ignore_index = True)

if df_courses == None:
    df_courses = pd.DataFrame(language[1])
else:
    df_courses = df_courses.append(language[1], ignore_index = True)

if df_lessons == None:
    df_lessons = pd.DataFrame(language[2])
else:
    df_lessons = df_lessons.append(language[2], ignore_index = True)
    

C:\Users\kevin\AppData\Local\Temp\ipykernel_24660\484113438.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driver_path)


['Communication', 'Startup / Innovate', 'Finance', 'Marketing', 'Operations']
['https://www.life-global.org/fr/categorylist/1-communication', 'https://www.life-global.org/fr/categorylist/2-startup---innovate', 'https://www.life-global.org/fr/categorylist/3-finance', 'https://www.life-global.org/fr/categorylist/4-marketing', 'https://www.life-global.org/fr/categorylist/5-operations']


In [18]:
df_courses

,Name,Duration,Pace,Language,Author,Code,Description,Category,Link
0,Des présentations efficaces,Pas plus de 60 minutes,À votre rythme,Français,HP LIFE,HPL-FR10,Comment puis-je créer un diaporama de présenta...,Communication,https://www.life-global.org/fr/fr/course/127-d...
1,Présenter des données,Pas plus de 60 minutes,À votre rythme,Français,HP LIFE,HPL-FR14,Quel est le moyen le plus efficace de présente...,Communication,https://www.life-global.org/fr/fr/course/125-p...
2,Les e-mails professionnels,Pas plus de 60 minutes,À votre rythme,Français,HP LIFE,HPL-FR11,Comment puis-je utiliser efficacement les e-ma...,Communication,https://www.life-global.org/fr/fr/course/126-l...
3,Communications professionnelles,Pas plus de 60 minutes,À votre rythme,Français,HP LIFE,HPL-FR26,Comment améliorer mes communications professio...,Communication,https://www.life-global.org/fr/fr/course/123-c...
4,Leadership efficace,Pas plus de 60 minutes,À votre rythme,Français,HP LIFE,HPL-FR23,Comment puis-je devenir un leader plus efficac...,Communication,https://www.life-global.org/fr/fr/course/124-l...
5,Économie circulaire,Pas plus de 60 minutes,À votre rythme,Français,HP LIFE,HPL-FR33,Comment puis-je concevoir mon entreprise afin ...,Créer / Innover,https://www.life-global.org/fr/fr/course/337-%...
6,Démarrer une petite entreprise,Pas plus de 60 minutes,À votre rythme,Français,HP LIFE,HPL-FR28,Comment puis-je créer une petite entreprise ?\...,Créer / Innover,https://www.life-global.org/fr/fr/course/114-d...
7,L'entrepreneuriat social,Pas plus de 60 minutes,À votre rythme,Français,HP LIFE,HPL-FR21,"Comment puis-je créer une entreprise durable, ...",Créer / Innover,https://www.life-global.org/fr/fr/course/323-l...
8,Moteurs de croissance,Pas plus de 60 minutes,À votre rythme,Français,HP LIFE,HPL-FR31,Qu’est-ce qu’un moteur de croissance et commen...,Créer / Innover,https://www.life-global.org/fr/fr/course/111-m...
9,Impression 3D,Pas plus de 60 minutes,À votre rythme,Français,HP LIFE & Youth for Technology Foundation\r\n ...,HPL-FR29,Comment puis-je développer un site Internet po...,Créer / Innover,https://www.life-global.org/fr/fr/course/113-i...


In [21]:
language =  getAll(languageAll[0], languageBase[0])
df1 = df_categories
df2 = df_courses
df3 = df_lessons

new = pd.DataFrame(language[0], columns=['Name'])
df_categories = df_categories.append(new, ignore_index = True)

df_courses = df_courses.append(language[1], ignore_index = True)

df_lessons = df_lessons.append(language[2], ignore_index = True)

C:\Users\kevin\AppData\Local\Temp\ipykernel_24660\3730349741.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_categories = df_categories.append(new, ignore_index = True)
C:\Users\kevin\AppData\Local\Temp\ipykernel_24660\3730349741.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_courses = df_courses.append(language[1], ignore_index = True)
C:\Users\kevin\AppData\Local\Temp\ipykernel_24660\3730349741.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_lessons = df_lessons.append(language[2], ignore_index = True)


In [26]:
language =  getAll(languageAll[2], languageBase[2])

df1 = df_categories
df2 = df_courses
df3 = df_lessons

new = pd.DataFrame(language[0], columns=['Name'])
df_categories = df_categories.append(new, ignore_index = True)

df_courses = df_courses.append(language[1], ignore_index = True)

df_lessons = df_lessons.append(language[2], ignore_index = True)

C:\Users\kevin\AppData\Local\Temp\ipykernel_24660\484113438.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driver_path)


['الاتصالات', 'المشاريع الناشئة / الابتكار', 'المالية', 'تسويق', 'عمليات']
['https://www.life-global.org/ar/ar/categorylist/1-%D8%A7%D9%84%D8%A7%D8%AA%D8%B5%D8%A7%D9%84%D8%A7%D8%AA', 'https://www.life-global.org/ar/ar/categorylist/2-%D8%A7%D9%84%D9%85%D8%B4%D8%A7%D8%B1%D9%8A%D8%B9-%D8%A7%D9%84%D9%86%D8%A7%D8%B4%D8%A6%D8%A9---%D8%A7%D9%84%D8%A7%D8%A8%D8%AA%D9%83%D8%A7%D8%B1', 'https://www.life-global.org/ar/ar/categorylist/3-%D8%A7%D9%84%D9%85%D8%A7%D9%84%D9%8A%D8%A9', 'https://www.life-global.org/ar/ar/categorylist/4-%D8%AA%D8%B3%D9%88%D9%8A%D9%82', 'https://www.life-global.org/ar/ar/categorylist/5-%D8%B9%D9%85%D9%84%D9%8A%D8%A7%D8%AA']


C:\Users\kevin\AppData\Local\Temp\ipykernel_24660\3063139637.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_categories = df_categories.append(new, ignore_index = True)
C:\Users\kevin\AppData\Local\Temp\ipykernel_24660\3063139637.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_courses = df_courses.append(language[1], ignore_index = True)
C:\Users\kevin\AppData\Local\Temp\ipykernel_24660\3063139637.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_lessons = df_lessons.append(language[2], ignore_index = True)


In [27]:
language =  getAll(languageAll[3], languageBase[3])

df1 = df_categories
df2 = df_courses
df3 = df_lessons

new = pd.DataFrame(language[0], columns=['Name'])
df_categories = df_categories.append(new, ignore_index = True)

df_courses = df_courses.append(language[1], ignore_index = True)

df_lessons = df_lessons.append(language[2], ignore_index = True)

C:\Users\kevin\AppData\Local\Temp\ipykernel_24660\484113438.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driver_path)


['संप्रेषण', 'स्टार्टअप / नवाचार', 'वित्त', 'विपणन', 'प्रचालन']
['https://www.life-global.org/hi/hi/categorylist/1-%E0%A4%B8%E0%A4%82%E0%A4%AA%E0%A5%8D%E0%A4%B0%E0%A5%87%E0%A4%B7%E0%A4%A3', 'https://www.life-global.org/hi/hi/categorylist/2-%E0%A4%B8%E0%A5%8D%E0%A4%9F%E0%A4%BE%E0%A4%B0%E0%A5%8D%E0%A4%9F%E0%A4%85%E0%A4%AA---%E0%A4%A8%E0%A4%B5%E0%A4%BE%E0%A4%9A%E0%A4%BE%E0%A4%B0', 'https://www.life-global.org/hi/hi/categorylist/3-%E0%A4%B5%E0%A4%BF%E0%A4%A4%E0%A5%8D%E0%A4%A4', 'https://www.life-global.org/hi/hi/categorylist/4-%E0%A4%B5%E0%A4%BF%E0%A4%AA%E0%A4%A3%E0%A4%A8', 'https://www.life-global.org/hi/hi/categorylist/5-%E0%A4%AA%E0%A5%8D%E0%A4%B0%E0%A4%9A%E0%A4%BE%E0%A4%B2%E0%A4%A8']


C:\Users\kevin\AppData\Local\Temp\ipykernel_24660\4072615141.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_categories = df_categories.append(new, ignore_index = True)
C:\Users\kevin\AppData\Local\Temp\ipykernel_24660\4072615141.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_courses = df_courses.append(language[1], ignore_index = True)
C:\Users\kevin\AppData\Local\Temp\ipykernel_24660\4072615141.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_lessons = df_lessons.append(language[2], ignore_index = True)


In [31]:
language =  getAll(languageAll[4], languageBase[4])

df1 = df_categories
df2 = df_courses
df3 = df_lessons

new = pd.DataFrame(language[0], columns=['Name'])
df_categories = df_categories.append(new, ignore_index = True)

df_courses = df_courses.append(language[1], ignore_index = True)

df_lessons = df_lessons.append(language[2], ignore_index = True)

C:\Users\kevin\AppData\Local\Temp\ipykernel_24660\484113438.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driver_path)


['Comunicação', 'Iniciar / Inovar', 'Finanças', 'Marketing', 'Operações']
['https://www.life-global.org/pt/pt/categorylist/1-comunica%C3%A7%C3%A3o', 'https://www.life-global.org/pt/pt/categorylist/2-iniciar---inovar', 'https://www.life-global.org/pt/pt/categorylist/3-finan%C3%A7as', 'https://www.life-global.org/pt/pt/categorylist/4-marketing', 'https://www.life-global.org/pt/pt/categorylist/5-opera%C3%A7%C3%B5es']


C:\Users\kevin\AppData\Local\Temp\ipykernel_24660\2741681360.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_categories = df_categories.append(new, ignore_index = True)
C:\Users\kevin\AppData\Local\Temp\ipykernel_24660\2741681360.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_courses = df_courses.append(language[1], ignore_index = True)
C:\Users\kevin\AppData\Local\Temp\ipykernel_24660\2741681360.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_lessons = df_lessons.append(language[2], ignore_index = True)


In [36]:
df_categories
df_lessons
df_courses

,Name,Duration,Pace,Language,Author,Code,Description,Category,Link
0,Des présentations efficaces,Pas plus de 60 minutes,À votre rythme,Français,HP LIFE,HPL-FR10,Comment puis-je créer un diaporama de présenta...,Communication,https://www.life-global.org/fr/fr/course/127-d...
1,Présenter des données,Pas plus de 60 minutes,À votre rythme,Français,HP LIFE,HPL-FR14,Quel est le moyen le plus efficace de présente...,Communication,https://www.life-global.org/fr/fr/course/125-p...
2,Les e-mails professionnels,Pas plus de 60 minutes,À votre rythme,Français,HP LIFE,HPL-FR11,Comment puis-je utiliser efficacement les e-ma...,Communication,https://www.life-global.org/fr/fr/course/126-l...
3,Communications professionnelles,Pas plus de 60 minutes,À votre rythme,Français,HP LIFE,HPL-FR26,Comment améliorer mes communications professio...,Communication,https://www.life-global.org/fr/fr/course/123-c...
4,Leadership efficace,Pas plus de 60 minutes,À votre rythme,Français,HP LIFE,HPL-FR23,Comment puis-je devenir un leader plus efficac...,Communication,https://www.life-global.org/fr/fr/course/124-l...
...,...,...,...,...,...,...,...,...,...
160,Eficiência Energética: Fazer Mais por Menos,Não mais de 60 minutos,No seu próprio ritmo,\tPortuguês,HP LIFE,HPL-PT22,Como posso poupar energia em meu negócio?\nApr...,Operações,https://www.life-global.org/pt/pt/course/62-ef...
161,TI para o Sucesso nos Negócios,Não mais de 60 minutos,No seu próprio ritmo,\tPortuguês,HP LIFE,HPL-PT24,Como avalio qual tecnologia devo usar em meu n...,Operações,https://www.life-global.org/pt/pt/course/63-ti...
162,Gestão de Estoque,Não mais de 60 minutos,No seu próprio ritmo,\tPortuguês,HP LIFE,HPL-PT20,Quanto preciso vender para ter lucro?\nAprenda...,Operações,https://www.life-global.org/pt/pt/course/61-ge...
163,Maximização da Capacidade,Não mais de 60 minutos,No seu próprio ritmo,\tPortuguês,HP LIFE,HPL-PT05,Como posso maximizar o volume de produtos e se...,Operações,https://www.life-global.org/pt/pt/course/58-ma...


In [37]:
df_categories.to_csv("CategoriesHP.csv",encoding='utf-8', index=False)
df_courses.to_csv("CoursesHP.csv",encoding='utf-8', index=False)
df_lessons.to_csv("LessonsHP.csv",encoding='utf-8', index=False)